In [ ]:
import requests
import os
from datetime import datetime
from pathlib import Path
import pyarrow
import geopandas as gpd 
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.ops import unary_union
import contextily as cx
import seaborn as sns
from datetime import datetime

root_dir = Path("~/Desktop/Desktop/epidemiology_PhD/00_repos/").expanduser()

url = "https://hub.arcgis.com/api/v3/datasets/025fb2ea05f14890b2b11573341b5b18_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1"

output_dir = root_dir / "los_angeles_2025_fire_disasters_exp/data/calfire_boundaries"
data_dir = root_dir / "los_angeles_2025_fire_disasters_exp/data"

so_cal_counties = ["025", "029", "037", "065", "059", "071", "073", "083", "111", "079"]

crs = 26911


In [ ]:
# outline 
# goal: zcta exposure dataset with exposure (yes/no) based on various buffers (specified below) 
# steps:
    # 1. download fires data
    # 2. load census tract data
    # 3. load wf boundary data
    # 4. create merged fire polys (agg over days)
    # 5. buffer the fire polys (buffers: .5, 1, 10, 20 km)
    # 6. overlay each buffer version with census tracts to determine if there is any overlap (any overlap at all = exposed)
    # 7. final dataset: census_tract, exposed_0.5km, exposed_1km, exposed_10km, exposed_20km

In [ ]:
# step 1: download fires data

# generate a filename with the current date
filename = f"data_{datetime.now().strftime('%Y_%m_%d')}.geojson"
output_path = os.path.join(output_dir, filename)

# Download the file
response = requests.get(url)
# Check if the request was successful
response.raise_for_status()  

# Save
with open(output_path, "wb") as file:
    file.write(response.content)

In [ ]:
# steps 2-3: load and clean fires and cts data 

# data contains wf data going back to 2024 or 2023 
# filter to dates since January 7th, 2025 and take out time
fires = gpd.read_file(output_path).to_crs(epsg=crs)
cts = gpd.read_file(data_dir / "tl_2010_06_tract10.shp").to_crs(epsg=crs)
cts = cts[['geometry', 'GEOID10', 'COUNTYFP10']]
cts = cts[cts['COUNTYFP10'].isin(so_cal_counties)]

zctas = gpd.read_file(data_dir / "tl_2020_us_zcta520.shp").to_crs(epsg=crs)
zctas = zctas[['geometry', 'ZCTA5CE20']].rename(columns={'ZCTA5CE20': 'zcta'})
zctas_socal = gpd.sjoin(zctas, cts, how='inner', predicate='intersects')
zctas_socal = zctas_socal.drop_duplicates(subset='zcta')

fires["poly_DateCurrent"] = fires["poly_DateCurrent"].dt.tz_convert('US/Pacific')
fires = fires[fires['poly_DateCurrent'] > '2025-01-06']
fires["poly_DateCurrent"] = fires["poly_DateCurrent"].dt.date

# fill in names for all fires 
fires["incident_name"] = fires["incident_name"].str.lower()
fires['incident_name'].fillna(fires['mission'].str.split('-').str[2].str.lower(), inplace=True)

# filter to only the cols we need
# incident_name, poly_DateCurrent, geometry
fires = fires[["incident_name", "poly_DateCurrent", "geometry"]]

# NOTE: there is one row that is called kenneth but has a poly that covers both pallisades and kenneth. from jan 9
# leaving it for now!

In [ ]:
# step 4: create merged fire polys (agg over days)
# step 5: buffer the fire polys (buffers: .5, 1, 10, 20 km)
# step 6: overlay each buffer version with census tracts to determine if there is any overlap (any overlap at all = exposed)
# step 7: final dataset: census_tract, exposed_0.5km, exposed_1km, exposed_10km, exposed_20km
buffer_distances = [0.5, 1, 10, 20]
buffers = [500, 1_000, 10_000, 20_000]

# Create empty dictionary to store results
zcta_exposures = {}
# Create list to store summary table
summary_data = []

# For each buffer distance
for dist, dist_km in zip(buffers, buffer_distances):
    # Create buffer
    buffered_fires = fires.geometry.buffer(dist)
    
    # Dissolve all fire buffers into single geometry
    combined_buffer = buffered_fires.unary_union

    # Count exposed tracts
    exposed_zctas = zctas_socal[zctas_socal.geometry.intersects(combined_buffer)]
    num_exposed = len(exposed_zctas)
    print(f"Buffer distance: {dist_km} km, Number of exposed ZCTAs: {num_exposed}")

    # Store in our list
    summary_data.append({
        'buffer_distance_km': dist_km,
        'num_exposed_zctas': num_exposed
    })
        
    # Store zctas for this buffer distance
    zcta_exposures[f'exposed_{dist_km}buffer'] = exposed_zctas['zcta'].tolist()

# Create final dataframe
result_df = pd.DataFrame({'zcta': zctas_socal['zcta'].values})

# Add exp cols
for col in zcta_exposures.keys():
    result_df[col] = result_df['zcta'].isin(zcta_exposures[col]).astype(int)

# Create summary dataframe and list of unique zctas
summary_df = pd.DataFrame(summary_data)
unique_zctas = result_df['zcta'].unique() # delete ?

# Write out data
result_df.to_csv(data_dir / f"zcta_exposures_{datetime.now().strftime('%Y_%m_%d')}.csv", index=False)
summary_df.to_csv(data_dir / f"zcta_summary_exposures_{datetime.now().strftime('%Y_%m_%d')}.csv", index=False)
pd.DataFrame(unique_zctas, columns=['zcta']).to_csv(data_dir / f"zctas_{datetime.now().strftime('%Y_%m_%d')}.csv", index=False) # delete? 

In [ ]:
# add percents to joans summary file (denom = zips that overlap socal counties)